<a href="https://colab.research.google.com/github/jared-ni/6.8610-project/blob/main/claude_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets
!pip install spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz
!pip install googletrans==4.0.0-rc1
!pip install deep-translator
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.6.1-py2.py3-n

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Create a DataFrame
df = pd.DataFrame({'Name': ['John', 'Mary'], 'Age': [25, 30]})

# Write to CSV
df.to_csv('/content/drive/My Drive/dummy_data.csv', index=False)

In [5]:
from google.colab import userdata
claude_token = userdata.get('claude_token')

In [6]:
import csv
from datasets import load_dataset
import pandas as pd
import spacy
from deep_translator import GoogleTranslator
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm

In [7]:
# Flags for which LLMs to use
USE_LLAMA = True
USE_MISTRAL = False
USE_FALCON = False

# Define max_length multiplier for LLM prompts
MAX_LENGTH_MULTIPLIER = 2

# Define k hyperparameter
K_HYPERPARAMETER = 3

In [23]:
# Load datasets into pandas DataFrames
def load_law_dataset():
    ds = load_dataset("casehold/casehold", "all")
    train_df = pd.DataFrame(ds['train'])
    test_df = pd.DataFrame(ds['test'])
    validation_df = pd.DataFrame(ds['validation'])
    law_dataset = pd.concat([train_df, test_df, validation_df], ignore_index=True)['citing_prompt']
    return law_dataset

def load_medical_dataset():
    ds = load_dataset("zhengyun21/PMC-Patients")
    train_df = pd.DataFrame(ds['train'])
    medical_dataset = train_df['patient']
    return medical_dataset

# Combine datasets
def load_all_datasets():
    law_dataset = load_law_dataset()
    medical_dataset = load_medical_dataset()
    return [law_dataset, medical_dataset]

In [24]:
# Load SpaCy model
def load_spacy_model(model_path='en_core_sci_sm'):
    return spacy.load(model_path)

# Extract entities from text
def extract_entities(nlp, text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents]

# Translate entities to a target language
def translate_entities(entities, target_lang):
    translations = [GoogleTranslator(source='auto', target=target_lang).translate(entity) for entity in entities]
    return translations

In [72]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.5 MB/s eta 0:00:00


In [83]:
from itertools import combinations
import jieba
import sacrebleu

# Calculate JTC score
def calculate_JTC(translations, text, entities):
    jtc_score = 0
    n = len(entities)
    entity_counts = 0

    for entity, translated_entity in entities.items():
        # Count occurrences of the entity in the original text
        c = text.count(entity)
        entity_counts += c
        if c == 0 or not translated_entity:
            continue  # Skip entities that are not present or have no translation
        jargon_inconsistency = K_HYPERPARAMETER * c

        for translation in translations:
            # Count occurrences of the translated entity in the translation
            t = translation.count(translated_entity)
            # Calculate penalty for mismatched occurrences
            jargon_inconsistency -= t

        # Update the JTC score
        jtc_score += jargon_inconsistency

    # Normalize and invert the score
    normalized_score = jtc_score / max(K_HYPERPARAMETER * entity_counts, 1)
    return 1 - normalized_score

# calculates the Jaccard similarity between the k translations
def calculate_jaccard(translations, target_lang):
    jaccard_scores = []
    if target_lang == "Simplified Chinese":
        translations = [set(jieba.lcut(translation)) for translation in translations]
    else:
        translations = [set(translation.split()) for translation in translations]
    # print("translations:", translations)
    for t1, t2 in combinations(translations, 2):
        # Calculate Jaccard similarity for the pair
        intersection = len(t1.intersection(t2))
        union = len(t1.union(t2))
        jaccard_score = intersection / union if union > 0 else 0
        jaccard_scores.append(jaccard_score)
    # print("Jaccard Scores:", jaccard_scores)
    # Return the average Jaccard similarity
    return sum(jaccard_scores) / len(jaccard_scores) if jaccard_scores else 0

# chrF++ ⇒ google translate serves as a reference ground truth
def calculate_chrf(ground_truth_translation, translations, n_value=6):
    ground_truth_translations = [ground_truth_translation] * len(translations)
    # sacrebleu.corpus_chrf calculates chrF++ directly
    chrf = sacrebleu.corpus_chrf(
        translations,  # List of translated texts
        ground_truth_translations,  # List of reference texts
        beta=2  # Default beta for F-score weighting
    )
    return chrf.score

In [12]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have h

In [19]:
# import anthropic

# api_key = userdata.get('claude_token')
# client = anthropic.Anthropic(api_key=api_key)

# message = client.messages.create(
#     # model="claude-3-5-sonnet-20241022",
#     model="claude-3-haiku-20240307",
#     max_tokens=512,
#     messages=[
#         {"role": "user", "content": "Please translate this to Chinese: The patient was diagnosed with diabetes and is very sick"}
#     ]
# )
# print(message.content)

[TextBlock(text='该患者被诊断患有糖尿病,病情非常严重。', type='text')]


In [47]:
import anthropic

# Helper function to interact with Claude
def invoke_claude(prompt, model="claude-3-haiku-20240307", max_tokens=512):
    api_key = userdata.get('claude_token')
    client = anthropic.Anthropic(api_key=api_key)
    response = client.messages.create(
        model=model,
        max_tokens=max_tokens,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text

In [80]:
from tqdm import tqdm

In [81]:
# Main function for the experiment
def run_pipeline_claude(target_lang, dataset_index=0, results_file_in_drive="results.csv"):
    datasets = load_all_datasets()
    lang_abbrs = {"Simplified Chinese": "zh-CN", "French": "fr"}

    # Path to the results file in Google Drive
    results_path = f'/content/drive/My Drive/{results_file_in_drive}'

    # Initialize the CSV file with headers
    with open(results_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Text Index", "Regular_JTC_Score", "LEAP_JTC_Score",
                         "Regular_Jaccard_Score", "LEAP_Jaccard_Score",
                         "Regular_chrF", "LEAP_chrF"])

    dataset = datasets[dataset_index]
    dataset_name = "law" if dataset_index == 0 else "medical"
    print(f"Processing dataset: {dataset_name}")

    for i, text in enumerate(tqdm(dataset[:3], desc=f"Processing {dataset_name} dataset")):
        text = " ".join(text.split()[:50])  # Truncate to the first 50 words
        # print(f"Text {i+1}: {text}")

        named_entities = list(set(extract_entities(load_spacy_model(), text)))
        # print("Named entities:", named_entities)

        # Translate named entities to the target language
        named_entities_translations = translate_entities(named_entities, lang_abbrs[target_lang])
        # print("Translations:", named_entities_translations)
        named_entity_mapping = {e: t for e, t in zip(named_entities, named_entities_translations)}

        # Regular text translation
        regular_translations = []
        for k in range(3):  # Adjust K_HYPERPARAMETER as needed
            prompt = f"Please return only the translation and nothing else. Translate the following text to {target_lang}: {text}"
            response = invoke_claude(prompt)
            # print("\nGenerated Response (Regular):", response)
            regular_translations.append(response)

        # LEAP text translation
        leap_translations = []
        for k in range(3):  # Adjust K_HYPERPARAMETER as needed
            prompt = f"Please return only the translation and nothing else. Translate the following text into {target_lang} using these mappings {str(named_entities_translations)}: {text}"
            response = invoke_claude(prompt)
            # print("\nGenerated Response (LEAP):", response)
            leap_translations.append(response)

        # Calculate JTC scores
        regular_jtc_score = calculate_JTC(regular_translations, text, named_entity_mapping)
        leap_jtc_score = calculate_JTC(leap_translations, text, named_entity_mapping)

        # Calculate Jaccard Similarities
        regular_jaccard_score = calculate_jaccard(regular_translations, target_lang)
        leap_jaccard_score = calculate_jaccard(leap_translations, target_lang)

        # chrF++ Metric
        ground_truth_translation = GoogleTranslator(source='auto', target=lang_abbrs[target_lang]).translate(text)
        regular_chrf = calculate_chrf(ground_truth_translation, regular_translations)
        leap_chrf = calculate_chrf(ground_truth_translation, leap_translations)

        # Append results to the file in Google Drive
        with open(results_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            row = [i + 1, regular_jtc_score, leap_jtc_score,
                   regular_jaccard_score, leap_jaccard_score,
                   regular_chrf, leap_chrf]
            writer.writerow(row)
            print(f"Appended result to Google Drive file: {row}")

In [84]:
# Run the pipeline for different target languages and datasets
target_langs = ["Simplified Chinese", "French"]
dataset_indices = [0, 1]  # 0: Law, 1: Medical

# Outer loop for target languages
for target_lang in tqdm(target_langs, desc="Target Languages"):
    # Inner loop for dataset indices
    for dataset_index in tqdm(dataset_indices, desc=f"Datasets for {target_lang}", leave=False):
        results_file = f"full_claude_{target_lang}_{dataset_index}_results.csv"
        run_pipeline_claude(target_lang=target_lang,
                            dataset_index=dataset_index,
                            results_file_in_drive=results_file)

Datasets for Simplified Chinese:   0%|          | 0/2 [00:00<?, ?it/s]

Processing dataset: law




Processing law dataset:   0%|          | 0/3 [00:00<?, ?it/s]

Processing law dataset:  33%|███▎      | 1/3 [00:11<00:23, 11.60s/it]

Appended result to Google Drive file: [1, 0.45833333333333337, 0.6875, 0.5151515151515151, 0.3847549909255898, 0.0, 0.0]




Processing law dataset:  67%|██████▋   | 2/3 [00:23<00:11, 11.55s/it]

...

Appended result to Google Drive file: [1, 0.7254901960784313, 0.9019607843137255, 0.85272397094431, 0.4202898550724638, 1.4450867052023122, 2.0891364902506964]




Processing medical dataset:  67%|██████▋   | 2/3 [00:21<00:10, 10.81s/it]

Appended result to Google Drive file: [2, 0.8541666666666666, 1.0, 0.8453889353012162, 1.0, 1.6, 1.7006802721088436]




Processing medical dataset: 100%|██████████| 3/3 [00:33<00:00, 11.32s/it]

Target Languages: 100%|██████████| 2/2 [04:04<00:00, 122.36s/it]

Appended result to Google Drive file: [3, 0.5614035087719298, 0.9122807017543859, 0.8472906403940886, 0.3974358974358975, 1.3623978201634876, 2.038043478260869]
